Homework 4 Statistics 489 Principles of Data Science and Statistics
----

Data Scraping, Cleaning, and Organization
----
Author: Carson Hanel

Prof: Alan Dabney

Import necessary packages:

In [13]:
from bs4 import BeautifulSoup
from dateutil import parser
import pandas as pd
import numpy as np
import requests

First, I'll build a list of sequential links from the website in order to scrub the first thousand reported bribes.
----

In [2]:
# Build the links for crawling
home  = "http://www.ipaidabribe.com/reports/all#gsc.tab=0"
left  = "http://www.ipaidabribe.com/reports/all?page="
right = "#gsc.tab=0"
mid   = np.arange(10, 1000, 10)
links = [home]
for x in mid:
    temp = str(left) + str(x) + str(right)
    links.append(temp)

Next, I'll iterate over the created URLs and extract the page data in HTML format, and convert to a list of BeautifulSoup objects. (This takes approx 30 minutes)
----

In [3]:
# Get all the pages for extraction
pages = []
for x in links:
    print("Processing page...")
    html = requests.get(str(x)).text
    soup = BeautifulSoup(html, 'html5lib')
    pages.append(soup)

Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing p

Next, scrape the gathered web pages for the required data.
----

In [14]:
# Extracting the titles from the bribes taken.
# This is only for a single website.
title      = []
date       = []
bribe      = []
department = []
views      = []
city       = []

for soup in pages:
    # Title of the offense.
    title_raw = soup.find_all("h3")
    for x in title_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        title.append(temp)
    
    # Date of the offense.
    date_raw   = soup.find_all("span", class_="date") 
    for x in date_raw:
        date.append(parser.parse(x.text))

    # Amount of bribe.
    bribe_raw = soup.find_all("li", class_="paid-amount")
    for x in bribe_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        temp = temp.split("INR")[-1]
        temp = temp.replace(",", "")
        bribe.append(int(temp))
    
    # Name of department
    department_raw = soup.find_all("li", class_="transaction")
    for x in department_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        department.append(temp)
    
    # Number of views
    views_raw = soup.find_all("li", class_="views")
    for x in views_raw:
        temp = x.text
        temp = temp.replace(" views", "")
        views.append(int(temp))
    
    # City
    city_raw = soup.find_all("a", class_="location")
    for x in city_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        temp = temp.replace(" ,", ",")
        city.append(temp)
    

Finally, put the scraped data into a Pandas Dataframe:
----

In [15]:
bribe_db = pd.DataFrame({'Title'     : title,
                         'Date'      : date,
                         'Bribe'     : bribe,
                         'Department': department,
                         'Views'     : views,
                         'City'      : city},
                       columns=['Date','Title','Views','Bribe','Department','City'])
bribe_db

,Date,Title,Views,Bribe,Department,City
0,2018-02-25,Traffic signal violation,58,100,Traffic Violations,"Chennai, Tamil Nadu"
1,2018-02-23,Barbie is being taken Due to verification,123,2000,Foreigner Registration and Verification,"New Delhi, Delhi"
2,2018-02-23,I entered no entry by mistake,145,200,Traffic Violations,"Bangalore, Karnataka"
3,2018-02-21,Charged 400INR saying pillion rider did not we...,258,400,Traffic Violations,"Bangalore, Karnataka"
4,2018-02-20,Had to pay Rs.150/- for a certificate of Rs.38/-,327,150,Marriage Certificate,"Bangalore, Karnataka"
5,2018-02-20,Traffic violation Rajarajeshwari Nagar,363,100,Traffic Violations,"Bangalore, Karnataka"
6,2018-02-19,Ek legal kam ko karbana k liye v risvat,407,200,Fitness Certificate,"Vidisha, Madhya Pradesh"
7,2018-02-19,To attain A TP from forest offucial,438,3000,Check Post,"Guwahati, Assam"
8,2018-02-18,Bribing maybe the only option,457,1000,Driving licence Process,"Bangalore, Karnataka"
9,2018-02-16,Asked For Bribe just for police verification (...,590,500,Police Verification for Passport,"Digboi, Assam"
